In [6]:
import numpy as np
import re
ch = ['chr'+str(i) for i in range(1,23)]
vcf_path = '/Users/ksg/gene_files/hg002/vcf/HG002_SVs_Tier1_v0.6副本.vcf'
write_auth_path = '/Users/ksg/gene_files/hg002/vcf/all_length/'

#---------------------------------------------------
pbhoney_path = r'/Users/ksg/gene_files/hg002/tools_result/pbhoney/'
pbsv_path = r'/Users/ksg/gene_files/hg002/tools_result/pbsv/'
#---------------------------------------------------


sniffles_path = r'/Users/ksg/gene_files/real/hg003/sniffles/' #/file_name
svim_path = r'/Users/ksg/gene_files/real/hg004/svim/' #/chr1-22
nextsv_path = '/Users/ksg/gene_files/real/hg003/nextsv/' #/file_name
smrtsv_path = '/Users/ksg/gene_files/real/hg004/smrtsv/' #/chr1-22
picky_path = ''
#ch = ['chr19','chr21','chr22']
print(ch)

['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8', 'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15', 'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22']


In [16]:
def get_authority_vcf_area(vcf_path,write_auth_path,ch):
    open_authority_vcf = open(vcf_path)
    authority_vcf = open_authority_vcf.readlines()
    open_authority_vcf.close()
    authority_vcf_list = [i.strip().split('\t') for i in authority_vcf]
    for chid in ch:
        write_file = open(write_auth_path+chid+'.txt',"w+")
        for i in authority_vcf_list:
            if chid[3:] == i[0]:
                if ('SVTYPE=DEL' in i[7]) and (len(i[3]) > 1):
                    if i[6] in ['LongReadHomRef', 'NoConsensusGT', 'ClusteredCalls', 'lt50bp', 'PASS']:
                        j = i[7].split('END=')[-1].split(';')[0]
                        if j[-1] == ']':
                            k = j[:-1]
                        else:
                            k = j
                        #if ((int(k)-int(i[1])) >= 50) and ((int(k)-int(i[1])) <= 1000):
                        write_file.write(chid+"\t"+i[1]+"\t"+k+"\t"+str(int(k)-int(i[1]))+"\n")
        write_file.close()

dd = get_authority_vcf_area(vcf_path,write_auth_path,ch)

In [16]:
#sniffles
def get_sniffles_area(sniffles_path,ch):
    for chid in ch:
        open_sniffles_result = open(sniffles_path+'hg004_hg19_'+chid+'.vcf')
        sniffles_list = [i.strip().split('\t') for i in open_sniffles_result.readlines()[34:]]
        open_sniffles_result.close()

        write_file = open(sniffles_path+'single_area/'+chid+'.txt',"w+")
        for j in sniffles_list:
            if (('SVTYPE=DEL' in j[7]) or (j[4] == '<DEL>')) and j[0] == chid and (j[3] not in ['N','.']):
                start = j[1]
                end = j[7].split(';')[3].split('=')[-1]
                #if int(end)-int(start)+1 >= 45 and int(end)-int(start)+1 <= 1100:
                write_file.write(start+'\t'+end+'\t'+str(int(end)-int(start)+1)+'\n')
        write_file.close()

            
sniffles_result = get_sniffles_area(sniffles_path,ch)

In [22]:
#svim
def get_svim_area(svim_path,ch):
    for chid in ch:
        open_svim_result = open(svim_path+chid+'/variants.vcf')
        svim_list = [i.strip().split('\t') for i in open_svim_result.readlines()[27:]]
        open_svim_result.close()

        write_file = open(svim_path+'single_area/'+chid+'.txt',"w+")
        for j in svim_list:
            #if ('=DEL' in j[7]) and j[6] == 'PASS' and j[0] == chid and int(j[5])>=5:
            if ('=DEL' in j[7]) and j[6] == 'PASS' and j[0] == chid and int(j[5])>=5:
                start = j[1]
                end = j[7].split(';')[1].split('=')[-1]
                #if int(end)-int(start)+1 >= 45 and int(end)-int(start)+1 <= 1100:
                write_file.write(start+'\t'+end+'\t'+str(int(end)-int(start)+1)+'\n')
        write_file.close()
            
svim_result = get_svim_area(svim_path,ch)

In [18]:
#nextsv
def get_nextsv_area(nextsv_path,ch):
    for chid in ch:
        open_nextsv_result = open(nextsv_path+'hg004_'+chid+'.minimap2.sniffles.vcf')
        nextsv_list = [i.strip().split('\t') for i in open_nextsv_result.readlines()[35:]]
        open_nextsv_result.close()
        #print(nextsv_list[:4])
        
        write_file = open(nextsv_path+'single_area/'+chid+'.txt',"w+")
        for j in nextsv_list:
            if (('SVTYPE=DEL' in j[7]) or (j[4] == '<DEL>')) and j[0] == chid and (j[3] not in ['N','.']):
                start = j[1]
                end = j[7].split(';')[3].split('=')[-1]
                #print(start,end)
                #if int(end)-int(start)+1 >= 45 and int(end)-int(start)+1 <= 1100:
                write_file.write(start+'\t'+end+'\t'+str(int(end)-int(start)+1)+'\n')
        write_file.close()
            
svim_result = get_nextsv_area(nextsv_path,ch)

In [7]:
#smrtsv
def get_smrtsv_area(smrtsv_path,ch):
    for chid in ch:
        try:
            open_smrtsv_result = open(smrtsv_path+chid+'/gaps_del.bed')
            smrtsv_list = [i.strip().split('\t') for i in open_smrtsv_result.readlines()]
            open_smrtsv_result.close()
        except FileNotFoundError:
            pass
    
        else:
            write_file = open(smrtsv_path+'single_area/'+chid+'.txt',"w+")
            for j in smrtsv_list:
                start = j[1]
                end = j[2]

                write_file.write(start+'\t'+end+'\t'+str(int(end)-int(start)+1)+'\n')
            write_file.close()
            
svim_result = get_smrtsv_area(smrtsv_path,ch)

In [12]:
ffd not in dff

True